# NBA Player Statistics Prediction - Inference

This notebook loads all trained models and makes predictions on the latest season features data.

## Models Used:
1. **Ridge Regression** (Linear model with regularization)
2. **XGBoost** (Gradient boosting)
3. **LightGBM** (Gradient boosting)
4. **Bayesian Multi-Output Regression** (PyMC with MatrixNormal)
5. **LSTM** (Deep learning with PyTorch)
6. **Ensemble Methods** (Simple averaging, weighted averaging, stacking)

## Input Data:
- `latest_season_features_for_inference.csv`: Features for the current season

## Output:
- Predictions for next season's statistics for all players

In [ ]:
import pandas as pd
import numpy as np
import joblib
import os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
RANDOM_SEED = 42

In [ ]:
def find_backend_dir(start_path=None):
    """
    Walk up directories from start_path (or cwd) until a folder named 'backend' is found.
    Returns the absolute path to the 'backend' folder.
    """
    if start_path is None:
        start_path = os.getcwd()
    curr_path = os.path.abspath(start_path)
    while True:
        # Check if 'backend' exists in this directory
        candidate = os.path.join(curr_path, "backend")
        if os.path.isdir(candidate):
            return candidate
        # If at filesystem root, stop
        parent = os.path.dirname(curr_path)
        if curr_path == parent:
            break
        curr_path = parent
    raise FileNotFoundError(f"No 'backend' directory found upward from {start_path}")

# Find the backend directory and CSV folder
backend_dir = find_backend_dir()
csv_dir = os.path.join(backend_dir, "CSVs")
models_dir = os.path.join(backend_dir, "Models")
output_dir = os.path.join(backend_dir, "Output")

print(f"Backend directory: {backend_dir}")
print(f"CSV directory: {csv_dir}")
print(f"Models directory: {models_dir}")
print(f"Output directory: {output_dir}")

In [ ]:
# Load the inference data
print("Loading inference data...")
inference_data = pd.read_csv(os.path.join(csv_dir, "latest_season_features_for_inference.csv"))

print(f"Inference data shape: {inference_data.shape}")
print(f"Columns: {len(inference_data.columns)}")
print(f"\nFirst few columns: {list(inference_data.columns[:10])}")
print(f"\nSample data:")
print(inference_data.head())

In [ ]:
# Load column information from different model files
print("Loading column information...")

# Try to load column info from different model files
column_info_sources = [
    'ridge_columns.joblib',
    'tree_models_columns.joblib',
    'bayesian_multioutput_columns.joblib',
    'lstm_columns.joblib'
]

feature_cols = None
target_cols = None

for source in column_info_sources:
    try:
        columns_info = joblib.load(os.path.join(models_dir, source))
        feature_cols = columns_info['feature_cols']
        target_cols = columns_info['target_cols']
        print(f"Loaded column info from {source}")
        break
    except Exception as e:
        print(f"Could not load from {source}: {e}")
        continue

if feature_cols is None or target_cols is None:
    # Fallback: infer columns from inference data
    feature_cols = [col for col in inference_data.columns if not col.startswith('next_') and col not in ['PERSON_ID', 'SEASON_ID']]
    target_cols = [col for col in inference_data.columns if col.startswith('next_')]
    print("Using fallback column inference")

print(f"\nFeature columns: {len(feature_cols)}")
print(f"Target columns: {len(target_cols)}")
print(f"\nTarget variables: {target_cols}")

In [ ]:
# Prepare inference data
print("Preparing inference data...")

# Select features
X_inference = inference_data[feature_cols].copy()

# Handle infinite values
X_inference.replace([np.inf, -np.inf], np.nan, inplace=True)

print(f"Inference features shape: {X_inference.shape}")
print(f"Missing values: {X_inference.isnull().sum().sum()}")

# Check if we have the required columns
missing_cols = set(feature_cols) - set(X_inference.columns)
if missing_cols:
    print(f"Warning: Missing columns: {missing_cols}")
    # Add missing columns with zeros
    for col in missing_cols:
        X_inference[col] = 0

# Ensure correct column order
X_inference = X_inference[feature_cols]

print(f"Final inference features shape: {X_inference.shape}")

## Load and Run Individual Models

We'll load each trained model and make predictions using their respective prediction functions.

In [ ]:
# Function to load model predictions using saved prediction functions
def load_model_predictions(model_name, X_inference, models_dir):
    """
    Load pre-trained model and get predictions using saved prediction functions
    
    Parameters:
    -----------
    model_name : str
        Name of the model
    X_inference : DataFrame
        Input features for inference
    models_dir : str
        Directory containing saved models
    
    Returns:
    --------
    predictions : DataFrame
        Model predictions
    """
    try:
        if model_name == 'ridge':
            # Load Ridge prediction function
            pred_func = joblib.load(os.path.join(models_dir, 'ridge_prediction_function.joblib'))
            predictions = pred_func(X_inference)
            
        elif model_name == 'xgboost':
            # Load XGBoost prediction function
            pred_func = joblib.load(os.path.join(models_dir, 'xgboost_prediction_function.joblib'))
            predictions = pred_func(X_inference)
            
        elif model_name == 'lightgbm':
            # Load LightGBM prediction function
            pred_func = joblib.load(os.path.join(models_dir, 'lightgbm_prediction_function.joblib'))
            predictions = pred_func(X_inference)
            
        elif model_name == 'bayesian':
            # Load Bayesian model
            import arviz as az
            trace = az.from_netcdf(os.path.join(models_dir, 'bayesian_multioutput_trace.nc'))
            scaler = joblib.load(os.path.join(models_dir, 'bayesian_multioutput_scaler.joblib'))
            imputer_X = joblib.load(os.path.join(models_dir, 'bayesian_multioutput_imputer_X.joblib'))
            
            # Preprocess data
            X_processed = pd.DataFrame(
                imputer_X.transform(X_inference),
                columns=X_inference.columns,
                index=X_inference.index
            )
            
            # Scale features
            X_scaled = scaler.transform(X_processed)
            
            # Get predictions
            beta_samples = trace.posterior['beta'].values
            intercept_samples = trace.posterior['intercept'].values
            
            # Make predictions
            pred = np.mean(np.dot(X_scaled, beta_samples) + intercept_samples, axis=(0, 1))
            
            # Convert to DataFrame
            predictions = pd.DataFrame(
                pred,
                columns=target_cols,
                index=X_inference.index
            )
            
        elif model_name == 'lstm':
            # Load LSTM model
            import torch
            
            # Load model info
            model_info = joblib.load(os.path.join(models_dir, 'lstm_model_info.joblib'))
            scaler_X = joblib.load(os.path.join(models_dir, 'lstm_scaler_X.joblib'))
            scaler_y = joblib.load(os.path.join(models_dir, 'lstm_scaler_y.joblib'))
            imputer_X = joblib.load(os.path.join(models_dir, 'lstm_imputer_X.joblib'))
            
            # Import LSTM model class and functions
            try:
                from lstm_training_testing import LSTMModel, create_sequences
            except ImportError:
                # If the module doesn't exist, define the functions here
                def create_sequences(X, y, sequence_length=5):
                    X_seq = []
                    y_seq = []
                    for i in range(sequence_length, len(X)):
                        X_seq.append(X.iloc[i-sequence_length:i].values)
                        y_seq.append(y.iloc[i].values if len(y) > 0 else np.zeros(len(target_cols)))
                    return np.array(X_seq), np.array(y_seq)
                
                class LSTMModel(torch.nn.Module):
                    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
                        super(LSTMModel, self).__init__()
                        self.hidden_size = hidden_size
                        self.num_layers = num_layers
                        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, 
                                                 batch_first=True, dropout=dropout if num_layers > 1 else 0)
                        self.fc1 = torch.nn.Linear(hidden_size, hidden_size // 2)
                        self.dropout = torch.nn.Dropout(dropout)
                        self.fc2 = torch.nn.Linear(hidden_size // 2, output_size)
                        self.relu = torch.nn.ReLU()
                    
                    def forward(self, x):
                        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
                        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
                        lstm_out, _ = self.lstm(x, (h0, c0))
                        lstm_out = lstm_out[:, -1, :]
                        out = self.relu(self.fc1(lstm_out))
                        out = self.dropout(out)
                        out = self.fc2(out)
                        return out
            
            # Create model
            model = LSTMModel(
                input_size=model_info['input_size'],
                hidden_size=model_info['hidden_size'],
                num_layers=model_info['num_layers'],
                output_size=model_info['output_size'],
                dropout=model_info['dropout']
            )
            
            # Load state dict
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            model.load_state_dict(torch.load(os.path.join(models_dir, 'lstm_best_model.pth'), map_location=device))
            model = model.to(device)
            model.eval()
            
            # Preprocess
            X_processed = pd.DataFrame(
                imputer_X.transform(X_inference),
                columns=X_inference.columns,
                index=X_inference.index
            )
            
            # Scale features
            X_scaled = scaler_X.transform(X_processed)
            
            # Create sequences
            sequence_length = model_info['sequence_length']
            X_seq, _ = create_sequences(pd.DataFrame(X_scaled), pd.DataFrame(), sequence_length)
            
            # Make predictions
            with torch.no_grad():
                X_tensor = torch.FloatTensor(X_seq).to(device)
                pred_scaled = model(X_tensor).cpu().numpy()
                pred = scaler_y.inverse_transform(pred_scaled)
                
                # Convert to DataFrame
                predictions = pd.DataFrame(
                    pred,
                    columns=target_cols,
                    index=X_inference.index[sequence_length:]
                )
        
        else:
            raise ValueError(f"Unknown model: {model_name}")
        
        return predictions
        
    except Exception as e:
        print(f"Error with {model_name} model: {e}")
        return None

In [ ]:
# Load and run all models
print("Loading and running all models...")

models = ['ridge', 'xgboost', 'lightgbm', 'bayesian', 'lstm']
predictions = {}

for model_name in models:
    print(f"\nRunning {model_name.upper()} model...")
    pred_df = load_model_predictions(model_name, X_inference, models_dir)
    
    if pred_df is not None:
        predictions[model_name] = pred_df
        print(f"  {model_name.upper()} predictions shape: {pred_df.shape}")
        print(f"  Sample predictions:")
        print(pred_df.head(3))
    else:
        print(f"  {model_name.upper()} failed to run")

print(f"\nSuccessfully ran {len(predictions)} models")

In [ ]:
# Create ensemble predictions
print("\nCreating ensemble predictions...")

if len(predictions) > 1:
    # Simple averaging
    pred_arrays = [predictions[model].values for model in predictions.keys()]
    simple_avg_pred = np.mean(pred_arrays, axis=0)
    
    simple_avg_df = pd.DataFrame(
        simple_avg_pred,
        columns=target_cols,
        index=X_inference.index
    )
    
    predictions['ensemble_simple'] = simple_avg_df
    
    # Weighted averaging (using equal weights for now)
    weights = np.ones(len(predictions)) / len(predictions)
    weighted_avg_pred = np.sum([weights[i] * pred_arrays[i] for i in range(len(pred_arrays))], axis=0)
    
    weighted_avg_df = pd.DataFrame(
        weighted_avg_pred,
        columns=target_cols,
        index=X_inference.index
    )
    
    predictions['ensemble_weighted'] = weighted_avg_df
    
    print(f"Created ensemble predictions")
    print(f"  Simple average shape: {simple_avg_df.shape}")
    print(f"  Weighted average shape: {weighted_avg_df.shape}")
else:
    print("Not enough models for ensemble")

print(f"\nTotal prediction methods: {len(predictions)}")

In [ ]:
# Analyze predictions
print("\nAnalyzing predictions...")

# Summary statistics for each model
for model_name, pred_df in predictions.items():
    print(f"\n{model_name.upper()} Predictions Summary:")
    print(f"  Shape: {pred_df.shape}")
    print(f"  Mean values:")
    for col in pred_df.columns:
        print(f"    {col}: {pred_df[col].mean():.3f}")
    print(f"  Standard deviations:")
    for col in pred_df.columns:
        print(f"    {col}: {pred_df[col].std():.3f}")

In [ ]:
# Visualize predictions
print("\nCreating visualizations...")

# Plot 1: Distribution of predictions for each target
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for i, target in enumerate(target_cols[:4]):  # Show first 4 targets
    if 'ensemble_simple' in predictions:
        axes[i].hist(predictions['ensemble_simple'][target], bins=30, alpha=0.7, label='Ensemble')
    axes[i].set_xlabel('Predicted Value')
    axes[i].set_ylabel('Frequency')
    axes[i].set_title(f'Distribution of {target} Predictions')
    axes[i].legend()

plt.tight_layout()
plt.show()

# Plot 2: Model comparison for one target
if len(predictions) > 1:
    target = target_cols[0]  # Use first target
    
    plt.figure(figsize=(12, 6))
    for model_name, pred_df in predictions.items():
        if target in pred_df.columns:
            plt.hist(pred_df[target], bins=30, alpha=0.6, label=model_name.upper())
    
    plt.xlabel('Predicted Value')
    plt.ylabel('Frequency')
    plt.title(f'Model Comparison for {target}')
    plt.legend()
    plt.show()

In [ ]:
# Save predictions
print("\nSaving predictions...")

# Create timestamp for file naming
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save individual model predictions
for model_name, pred_df in predictions.items():
    if not model_name.startswith('ensemble'):
        filename = f"{model_name}_predictions_{timestamp}.csv"
        filepath = os.path.join(output_dir, filename)
        pred_df.to_csv(filepath)
        print(f"  Saved {model_name} predictions to: {filepath}")

# Save ensemble predictions
if 'ensemble_simple' in predictions:
    ensemble_filename = f"ensemble_predictions_{timestamp}.csv"
    ensemble_filepath = os.path.join(output_dir, ensemble_filename)
    predictions['ensemble_simple'].to_csv(ensemble_filepath)
    print(f"  Saved ensemble predictions to: {ensemble_filepath}")

# Save all predictions together
all_predictions_filename = f"all_predictions_{timestamp}.joblib"
all_predictions_filepath = os.path.join(output_dir, all_predictions_filename)
joblib.dump(predictions, all_predictions_filepath)
print(f"  Saved all predictions to: {all_predictions_filepath}")

print("\nAll predictions saved successfully!")

In [ ]:
# Create summary report
print("\nCreating summary report...")

report = {
    'timestamp': timestamp,
    'input_data_shape': inference_data.shape,
    'feature_columns': len(feature_cols),
    'target_columns': len(target_cols),
    'models_used': list(predictions.keys()),
    'predictions_summary': {}
}

# Add summary statistics for each model
for model_name, pred_df in predictions.items():
    report['predictions_summary'][model_name] = {
        'shape': pred_df.shape,
        'mean_values': pred_df.mean().to_dict(),
        'std_values': pred_df.std().to_dict(),
        'min_values': pred_df.min().to_dict(),
        'max_values': pred_df.max().to_dict()
    }

# Save report
report_filename = f"inference_report_{timestamp}.joblib"
report_filepath = os.path.join(output_dir, report_filename)
joblib.dump(report, report_filepath)
print(f"  Saved inference report to: {report_filepath}")

# Print summary
print("\n" + "="*60)
print("INFERENCE SUMMARY REPORT")
print("="*60)
print(f"Timestamp: {timestamp}")
print(f"Input data shape: {inference_data.shape}")
print(f"Feature columns: {len(feature_cols)}")
print(f"Target columns: {len(target_cols)}")
print(f"Models used: {len(predictions)}")
print(f"\nModels:")
for model_name in predictions.keys():
    print(f"  - {model_name.upper()}")
print(f"\nOutput files saved to: {output_dir}")
print("="*60)

In [ ]:
# Create ensemble prediction function
def predict_with_ensemble(X, ensemble_type='weighted_avg', models_dir=models_dir):
    """
    Make predictions using ensemble methods
    
    Parameters:
    -----------
    X : DataFrame
        Input features
    ensemble_type : str
        Type of ensemble ('simple_avg', 'weighted_avg', 'stacking')
    models_dir : str
        Directory containing saved models
    
    Returns:
    --------
    predictions : DataFrame
        Ensemble predictions
    """
    # Load all individual models
    individual_predictions = {}
    
    for model_name in ['ridge', 'xgboost', 'lightgbm', 'bayesian', 'lstm']:
        try:
            pred_df = load_model_predictions(model_name, X, models_dir)
            if pred_df is not None:
                individual_predictions[model_name] = pred_df
        except Exception as e:
            print(f"Error loading {model_name}: {e}")
    
    if len(individual_predictions) == 0:
        raise ValueError("No models could be loaded")
    
    # Create ensemble
    if ensemble_type == 'simple_avg':
        pred_arrays = [individual_predictions[model].values for model in individual_predictions.keys()]
        ensemble_pred = np.mean(pred_arrays, axis=0)
    elif ensemble_type == 'weighted_avg':
        pred_arrays = [individual_predictions[model].values for model in individual_predictions.keys()]
        weights = np.ones(len(individual_predictions)) / len(individual_predictions)
        ensemble_pred = np.sum([weights[i] * pred_arrays[i] for i in range(len(pred_arrays))], axis=0)
    else:
        raise ValueError(f"Unknown ensemble type: {ensemble_type}")
    
    # Convert to DataFrame
    ensemble_df = pd.DataFrame(
        ensemble_pred,
        columns=target_cols,
        index=X.index
    )
    
    return ensemble_df

# Save the prediction function
prediction_func_path = os.path.join(models_dir, "ensemble_prediction_function.joblib")
joblib.dump(predict_with_ensemble, prediction_func_path)
print(f"Ensemble prediction function saved to: {prediction_func_path}")

print("\nInference pipeline completed successfully!")

## Inference Summary

The inference process has been completed successfully. Here's what was accomplished:

### Models Used:
- Ridge Regression (using saved prediction function)
- XGBoost (using saved prediction function)
- LightGBM (using saved prediction function)
- Bayesian Multi-Output Regression (PyMC)
- LSTM (PyTorch)
- Ensemble methods (Simple averaging, Weighted averaging)

### Key Features:
- **Multi-model predictions**: Each model provides its own predictions
- **Ensemble combinations**: Simple and weighted averaging of model predictions
- **Comprehensive output**: Individual model predictions and ensemble results
- **Detailed reporting**: Summary statistics and visualizations
- **Robust error handling**: Graceful handling of missing models

### Output Files:
- Individual model prediction CSV files
- Ensemble prediction CSV file
- All predictions in joblib format
- Detailed inference report
- Ensemble prediction function

### Next Steps:
1. Analyze the predictions for insights
2. Compare model performance on specific players
3. Validate predictions against actual performance
4. Use predictions for fantasy sports or scouting
5. Retrain models with new data as it becomes available